In [63]:
'''
/* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

select name 
FROM Facilities 
WHERE membercost !=0.0

/* Q2: How many facilities do not charge a fee to members? */

select count(name) 
FROM Facilities 
WHERE membercost =0.0

/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

select facid, name, membercost, monthlymaintenance 
From Facilities 
WHERE membercost < 0.2*monthlymaintenance

/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

SELECT * FROM `Facilities` 
WHERE facid= 1 or facid=5

/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

SELECT name, monthlymaintenance,
CASE WHEN monthlymaintenance > 100 THEN 'Expensive'
Else 'cheap' END As 'cheap/expensive'
FROM `Facilities`


/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */
SELECT * FROM `Members` 
ORDER BY joindate DESC

/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

SELECT DISTINCT concat (m.firstname,' ', m.surname) as 'full_name', f.name
FROM Bookings as b
INNER JOIN Facilities as f on b.facid = f.facid
INNER JOIN Members as m ON m.memid = b.memid
WHERE f.name LIKE 'Tennis Court%'
ORDER BY full_name

/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

SELECT
firstname AS member,
name AS facility,
cost
FROM(SELECT firstname,surname,name,
CASE WHEN firstname = 'GUEST' THEN guestcost * slots ELSE membercost * slots END AS cost,
starttime
FROM 
members
INNER JOIN bookings
ON members.memid = bookings.memid
INNER JOIN facilities
ON bookings.facid = facilities.facid) AS inner_table
WHERE starttime >= '2012-09-14' 
  AND starttime < '2012-09-15'
  AND cost > 30
ORDER BY cost DESC;

'''



"\n/* QUESTIONS \n/* Q1: Some of the facilities charge a fee to members, but some do not.\nWrite a SQL query to produce a list of the names of the facilities that do. */\n\nselect name \nFROM Facilities \nWHERE membercost !=0.0\n\n/* Q2: How many facilities do not charge a fee to members? */\n\nselect count(name) \nFROM Facilities \nWHERE membercost =0.0\n\n/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,\nwhere the fee is less than 20% of the facility's monthly maintenance cost.\nReturn the facid, facility name, member cost, and monthly maintenance of the\nfacilities in question. */\n\nselect facid, name, membercost, monthlymaintenance \nFrom Facilities \nWHERE membercost < 0.2*monthlymaintenance\n\n/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.\nTry writing the query without using the OR operator. */\n\nSELECT * FROM `Facilities` \nWHERE facid= 1 or facid=5\n\n/* Q5: Produce a list of facilities, with each labelled

In [6]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT Facilities.name,
        sum(CASE WHEN Bookings.memid!=0 Then Facilities.membercost*Bookings.slots
                ELSE Facilities.guestcost*Bookings.slots END) as revenue
                From Bookings INNER JOIN Facilities on Bookings.facid=Facilities.facid
                INNER JOIN Members on Members.memid=Bookings.Memid
                Group By Facilities.name Having revenue <1000
                Order by revenue DESC
                """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Pool Table', 270)
('Snooker Table', 240)
('Table Tennis', 180)


In [42]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT m2.firstname||m2.surname AS full, m1.firstname||m1.surname AS recommended
        From Members as m1
        Inner Join Members AS m2 On m1.memid=m2.recommendedby
        Order by full
                """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('AnnaMackenzie', 'DarrenSmith')
('AnneBaker', 'PonderStibbons')
('CharlesOwen', 'DarrenSmith')
('DavidJones', 'JaniceJoplette')
('DavidPinker', 'JemimaFarrell')
('DouglasJones', 'DavidJones')
('EricaCrumpet', 'TracySmith')
('FlorenceBader', 'PonderStibbons')
('GeraldButters', 'DarrenSmith')
('HenriettaRumney', 'MatthewGenting')
('HenryWorthington-Smyth', 'TracySmith')
('JackSmith', 'DarrenSmith')
('JaniceJoplette', 'DarrenSmith')
('JoanCoplin', 'TimothyBaker')
('JohnHunt', 'MillicentPurview')
('MatthewGenting', 'GeraldButters')
('MillicentPurview', 'TracySmith')
('NancyDare', 'JaniceJoplette')
('PonderStibbons', 'BurtonTracy')
('RamnareshSarwin', 'FlorenceBader')
('TimBoothe', 'TimRownam')
('TimothyBaker', 'JemimaFarrell')


In [50]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT Facilities.name,count(facid) from Bookings
        INNer Join Facilities using(facid)
        WHERE memid != 0
        group by facid
                """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Tennis Court 1', 308)
('Tennis Court 2', 276)
('Badminton Court', 344)
('Table Tennis', 385)
('Massage Room 1', 421)
('Massage Room 2', 27)
('Squash Court', 195)
('Snooker Table', 421)
('Pool Table', 783)


In [62]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT strftime('%m',starttime) as month,count(facid) from Bookings
        Inner Join Facilities Using(facid)
        Group by month
     
        
                """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('07', 658)
('08', 1472)
('09', 1913)
